In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141521
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-141521


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4, 
                                                            idle_seconds_before_scaledown=2400)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [76]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import normal, uniform, choice
from azureml.core import Environment
from azureml.core import ScriptRunConfig
import os

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn?view=azure-ml-py

# Specify parameter sampler
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#random-sampling
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.randomparametersampling?view=azure-ml-py
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451
# https://www.kaggle.com/joparga3/2-tuning-parameters-for-logistic-regression
ps = RandomParameterSampling({
    "C": choice(0.001, 0.01, 0.1, 1, 10, 100),
    "max_iter": choice(25,50,75,100)
})

# Specify a Policy
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#early-termination
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.banditpolicy?view=azure-ml-py#definition
policy = BanditPolicy(evaluation_interval=100, delay_evaluation=200, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.sklearn.sklearn?view=azure-ml-py
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')
src = ScriptRunConfig(source_directory='.',
                     script='./train.py',
                     compute_target=cpu_cluster,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.hyperdriveconfig?view=azure-ml-py
hd_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=1000,
                                    max_concurrent_runs=4)

In [77]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn?view=azure-ml-py
# https://azure-ml.southcentralus.instances.azureml.ms/examples/preview?example_id=tutorials%2Fimage-classification-mnist-data%2Fimg-classification-part1-training.ipynb

hd_run = exp.submit(config=hd_config)

RunDetails(hd_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [78]:
import joblib
# Get your best run and save the model from that run.
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#find-the-best-model
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn?view=azure-ml-py#save-and-register-the-model

best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
arguments = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print('\n Accuracy: ', best_run_metrics['Accuracy'])
print('\n C: ', arguments[1])
print('\n max_iter: ', arguments[3])

Best Run Id:  HD_9a754c49-9f33-422f-877f-a5727c0ea01a_1

 Accuracy:  0.9128429230395727

 C:  0.1

 max_iter:  75


In [79]:
# Save the model
model = best_run.register_model(model_name='hd-model',
                               model_path='./outputs/model.joblib',
                               tags={'Method':'Hyperdrive'},
                               properties={'Accuracy':best_run_metrics['Accuracy']})

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
azureml_ds = TabularDatasetFactory.from_delimited_files(path=web_path)
azureml_ds.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [4]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(azureml_ds)

# Split data into train and test
# # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [5]:
import numpy as np
import pandas as pd

#print('x_train:\n', x_train.shape)
#print('y_train:\n', y_train.shape)
#pd.DataFrame(x_train)
#pd.DataFrame(y_train)

# concatenate train data x and y and test
# https://pandas.pydata.org/docs/user_guide/merging.html
train_data = pd.concat([x_train, y_train], axis=1)
pd.DataFrame(train_data)

test_data = pd.concat([x_test, y_test], axis=1)
pd.DataFrame(test_data)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
28416,30,0,0,1,1,5,1,95,2,999,...,0,0,0,0,1,0,0,0,0,0
32162,47,1,0,0,0,5,3,106,1,999,...,1,0,0,0,0,0,0,1,0,0
29485,32,1,0,1,0,7,4,307,1,999,...,0,1,0,0,0,0,0,0,0,0
17997,32,0,0,1,0,6,2,45,2,999,...,1,0,0,0,0,0,0,1,0,0
21638,39,1,0,1,0,5,4,292,1,999,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17587,31,0,0,0,1,7,4,15,1,999,...,1,0,0,0,0,0,0,1,0,0
7766,26,0,0,1,0,7,5,200,3,999,...,1,0,0,0,1,0,0,0,0,0
30397,24,0,0,1,1,6,1,74,1,999,...,0,0,0,0,0,0,0,1,0,0
11422,77,0,0,1,0,11,1,445,2,999,...,0,1,0,0,0,0,0,0,0,1


In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
# from azureml.automl.runtime.onnx_convert import OnnxConverter
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py
# https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/classification-bank-marketing-all-features/auto-ml-classification-bank-marketing-all-features.ipynb
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    #n_cross_validations=10,
    validation_data=test_data
)

In [8]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_4db727bd-0b85-48cc-86c9-2da2a1cd5653

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positiv

In [64]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show() 

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [9]:
best_automl_run, fitted_automl_model = remote_run.get_output()
print(best_automl_run)
best_automl_run_metrics = best_automl_run.get_metrics()
print(best_automl_run_metrics['accuracy'])

Run(Experiment: udacity-project,
Id: AutoML_4db727bd-0b85-48cc-86c9-2da2a1cd5653_66,
Type: None,
Status: Completed)
0.9209759650400583


In [10]:
# Retrieve and save your best automl model.

best_automl_run, fitted_automl_model = remote_run.get_output()
best_automl_run_metrics = best_automl_run.get_metrics()

automl_model = remote_run.register_model(model_name = 'automl-model',
                                  tags={'Method':'AutoML'},
                                  description='AutoML Model trained on bank marketing data to predict if a client will subscribe to a term deposit')

In [71]:
# Delete ComputeTarget

cpu_cluster.delete()